<Table border="1">
<tr><td>
<img alt="INTRO Seguridad y Privacidad Python" title="INTRO Seguridad y Privacidad Python" src="./UAM+EPS_L.png" width="800">
</td><td>
</table>

## Tu turno 1: Probar el AES con el modo de operación CFB. Probar el AES con tamaños de claves superiores a 128 en modo CFB. Probar el DES3 con el modo de operación CFB.

Para probar la operación CFB basta cambiar el atributo AES.MODE_ECB por AES.MODE_CFB. A contiuación se encripta y desencripta el mensaje propuesto en la práctica.

In [5]:
from Crypto.Cipher import AES
from Crypto import Random
# Definición de la función lambda que comvierte una cadena a hexadecimal.
HEXA= lambda xx: ':'.join(hex(ord(x))[2:] for x in xx)

key = b'1111111111111111'
cipher = AES.new(key, AES.MODE_CFB)
msg = b'2222222222222222' # Mensaje de 16 bytes
print('Mensaje plano -->', msg)
# Cifrado
msgc = cipher.encrypt(msg)
print('Mensaje cifrado -->', msgc)

Mensaje plano --> b'2222222222222222'
Mensaje cifrado --> b'\xf44\x0er\xdc\xff\xeb$\xe8\x1b\xd4\xb2[\xc3\xb5Z'


In [6]:
cipher2 = AES.new(key, AES.MODE_CFB)

In [7]:
# Descifrado
cipher2.decrypt(cipher.encrypt(msg))
print('Mensaje descifrado -->', cipher2.decrypt(cipher.encrypt(msg)))

Mensaje descifrado --> b'2222222222222222'


A continuación, se cambia el número bytes de la clave respecto al ejemplo propuesto a 24 (AES-196) o 36 (AES-256).

doc:https://pycryptodome.readthedocs.io/en/latest/src/cipher/aes.html#:~:text=AES%20(Advanced%20Encryption%20Standard)%20is,block%20size%20of%2016%20bytes.&text=The%20secret%20key%20to%20use,192%20or%20AES%2D256).

In [8]:
from Crypto.Cipher import AES
from Crypto import Random
# Definición de la función lambda que comvierte una cadena a hexadecimal.
HEXA= lambda xx: ':'.join(hex(ord(x))[2:] for x in xx)

key = b'1'*24 #32
cipher = AES.new(key, AES.MODE_CFB)
msg = b'2222222222222222' # Mensaje de 16 bytes
print('Mensaje plano -->', msg)
# Cifrado
msgc = cipher.encrypt(msg)
print('Mensaje cifrado -->', msgc)

Mensaje plano --> b'2222222222222222'
Mensaje cifrado --> b'\r\xe8.\xb3\xa1\xf5y(F\xb1yX\xb6l8\xe6'


In [9]:
cipher2 = AES.new(key, AES.MODE_CFB)

In [10]:
# Descifrado
cipher2.decrypt(cipher.encrypt(msg))
print('Mensaje descifrado -->', cipher2.decrypt(cipher.encrypt(msg)))

Mensaje descifrado --> b'2222222222222222'


A continuación se prueba el DES3 reusando el código propuesto.

In [30]:
from Crypto.Cipher import DES3
from Crypto import Random
from Crypto.Random import get_random_bytes

iv = Random.new().read(DES3.block_size)
while True: # aseguramos la paridad de las subclaves para no degradar el método a un DES simple
    try:
        key = DES3.adjust_key_parity(get_random_bytes(24))
        break
    except ValueError:
        pass
    
print('clave: ', key)
cipher = DES3.new(key, DES3.MODE_CFB, iv)
plaintext = b'hola buenas'
msg = cipher.iv+cipher.encrypt(plaintext)
iv_sent = msg[:DES3.block_size] 
print('mensaje cifrado: ', msg)
cipher2 = DES3.new(key, DES3.MODE_CFB, iv_sent)
print('mensaje descifrado: ', cipher2.decrypt(msg[DES3.block_size:]))

clave:  b'\xf2y\x19d\xcbOk \x92p\xa1\xf1 \x0b\x01\x0b\x102=\x9d\xf4Q\x13\x9e'
mensaje cifrado:  b'\xce^\x1cU\xfa\x02g\x1a\xf48\xab\xf4\x16\x07Fu\xb8\x06\xe0'
mensaje descifrado:  b'hola buenas'


## Tu turno 2: Explicar con detalle porque para el cifrado de flujo RC4, como mostramos en el código anterior, la clave de largo plazo se combina complejamente con un Nonce a través de una función Hash ¿Porqué no se utiliza solo la clave de largo plazo? ¿Porqué no se utiliza ninguna función de Padding? ¿Qué relación tiene RC4 con OTP?

RC4 no toma un nonce o IV, por tanto, si se quiere reusar la clave (de largo plazo) para cifrar varios mensajes, debe crearse un nonce (independiente para cada nuevo mensaje) y combinar la clave de largo y el nonce para tener una nueva clave que se usará para cifrar finalmente el mensaje (de corto plazo). Puesto que en esta clave de corto plazo, siempre se repite una parte (la parte de la clave a largo plazo) la combinación debe realizarse con una función compleja, como es en este caso la función hash.


Esta función hash tiene como objetivo generar confusión entre la clave a largo y el nonce para que no se produzca una brecha de seguridad al conocer el atacante parte de la clave de encriptación (la que se corrresponde con la largo plazo que se esta reusando).

Las técnicas de padding consisten en añadir información difusa (frases sin sentido, bits adicionales, etc) al mensaje antes del cifrado para confundir a los atacantes y dificultar la búsqueda de patrones. No se añade una función padding porque no es un único mensaje lo que está en mayor riesgo, sino la clave de encriptación. Esta debilidad de RC4 viene de su algoritmo de programación de clave débil. Como el propósito es reusar la clave a largo plazo, no tiene sentido aplicar técnicas de padding a los mensajes, ya que una vez descubierta la clave largo plazo y con esfuerzo los nonce, se podrían descifrar todos los mensajes sin que el emisor lo sepa.

**La relación entre RC4 y OTP es que son algoritmos que integran la generación de números aleatorios para producir la clave de cifrado.** Cabe señalar que con estas técnicas no se recomienda reusar parcialmente las claves.

## Tu turno 3: Utiliza RSA para cifrar un mensaje pero invirtiendo el procedimiento del protocolo: primero utiliza la clave privada y luego la clave pública. Prueba que recuperas el mensaje ¿Explica que diferencia hay con el procedimiento anterior?

In [78]:
msg='Hola, quiero que me cifres con RSA' # mensaje a cifrar
# Importamos la clave pública para cifrar los datos
cifrar_rsa = PKCS1_OAEP.new(key)
# Importamos la clave privada para descifrar los datos
descifrar_rsa = PKCS1_OAEP.new(public_key)
# Ahora ciframos y desciframos
msgc = cifrar_rsa.encrypt(msg.encode()) # Sino se pone "b" en el mensaje, e deben codificar los datos a 'utf-8' ya que en En Python 3, 
                                        # no hay conversión implícita entre objetos Unicode (str) y objetos bytes.
print('msgc --> ' "%s"  % msgc) # Cifrado con la clave publica
print('msgc --> ' "%s"  % msgc.hex()) 
msg_descifrado = descifrar_rsa.decrypt(msgc) # Descifrado con la clave privada
#msg_descifrado = key.decrypt(msgc) # Descifrado con la clave privada
print('Mensaje decifrado --> ' "%s"  % msg_descifrado.decode())

msgc --> b'/\xbf,\xec\x1b\xe8Q\x87&J\xa5\xaf\xeeS\xae\x7f\x89\x84\xd3\x94X\xfav[V\x18\xf9\xc9\xdd\xd5\xa0\x95\xb5\xf6\xd8b\x9b\x02\xbb\x0f\x80\xb0K\xd8\xa6\x0bq\xbb\xf4\x82\xc6\x8a\n\x99\xefn\xc1\x9bhA\x97\x9c\x0b4dunwq\xdd\x92\x91\xa0\xff\x9b\xa9p\xb8\xea\xf3\x87W:\xc7\x18\x90\xe4eO\x10\xdb\xdf8\x02\x13\xbc\xf69\xa6C\x16A\xce!\x01\xdb\xd3}\xdcH#\x8eD\xeb\x95<\xe2$\xbb\xa8:N^\x13>\xed\xfbf'
msgc --> 2fbf2cec1be85187264aa5afee53ae7f8984d39458fa765b5618f9c9ddd5a095b5f6d8629b02bb0f80b04bd8a60b71bbf482c68a0a99ef6ec19b6841979c0b3464756e7771dd9291a0ff9ba970b8eaf387573ac71890e4654f10dbdf380213bcf639a6431641ce2101dbd37ddc48238e44eb953ce224bba83a4e5e133eedfb66


TypeError: This is not a private key

Al usar una clave privada para cifrar los datos y una pública para descifrarlo, tenemos una firma digital. Es decir, un mensaje cuyo contenido puede ser conocido por todos los remitentes, pero cuyo contenido no puede alterarse, ya que la clave pública dejaría de poder desencriptar el mensaje que se cifró con la clave privada.

**Por la implementación de Python, no se puede ejecutar de manera inversa. Se deben usar librerías específicas de firma digital.**

## Tu turno 4: Explicar porque Alice y Bob tienen una clave secreta común, k=53, cuando no se han pasado explicitamente la clave k=53.

Para entender como se ha calculado la clave, se debe ver el método Diffie-Hellman a detalle. 

El método de Diffie-Hellman es un método para intercambio de claves de manera segura.

Para implementar el método, Alice y Bob deben acordar dos enteros positivos $p, q$, de modo que $p$ es un número primo y $q$ es la base para las operaciones modulares, que debe ser raíz primitiva modulo $p$ (Un número $q$ es raíz primitiva modulo "p" si para todo entero "a" coprimo con "p" existe algun entero $b$ tal que $q^b \equiv a$ $mod$ $p$). Los números se escogen así para que se cumpla que la clave compartida $k \in [1, p-1]$

Una vez hecho esto, cada usuario elige su clave privada, digamos $a \in Z$ para Alice y $b\in Z$ para Bob, que no comparten con nadie. Con estos números, calculan sus claves públicas que compartiran a través del canal inseguro.

$$a^* = q^a \: \textrm{mod} \:p$$ 
$$b^* = q^b \: \textrm{mod} \:p$$

Hecho esto, y una vez ambos conocen la clave pública del otro, ambos pueden calcular la clave común usando las propiedades de la potenciación modular:

$$k_{\textrm{Alice}} = (b^*)^a \: mod \:p$$
$$k_{\textrm{Bob}} = (a^*)^b \: mod \:p$$

de manera que:

$$k_{\textrm{Alice}} = (b^*)^a \: mod \:p = (q^b \: \textrm{mod} \:p)^a \: mod \:p = (q^a \: \textrm{mod} \:p)^b \: mod \:p=
(a^*)^b \: mod \:p = k_{\textrm{Bob}}
$$


## Tu turno 5: Implementa el esquema Firma Digital y Confidencialidad:
<Table border="1">
<tr><td>
<img alt="INTRO Seguridad y Privacidad Python" title="INTRO Seguridad y Privacidad Python" src="./FirmayConfidencialidad.png" width="1000">
</td></tr>
</table>